# Jane Street: Neural Network Regression Starter

frok from yirun Zhang's https://www.kaggle.com/gogo827jz/jane-street-neural-network-starter

In [ ]:
import os, gc, datetime
from time import time

import numpy as np
import pandas as pd
import datatable as dtable
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

import tensorflow as tf
import tensorflow_addons as tfa
tf.random.set_seed(42)
import tensorflow.keras.backend as K
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Preprocessing

In [ ]:
print('Loading...')
train = dtable.fread('/kaggle/input/jane-street-market-prediction/train.csv').to_pandas()
features = [c for c in train.columns if 'feature' in c]

print('Filling...')
f_mean = train[features[1:]].mean()
train = train.query('weight > 0').reset_index(drop = True)
train = train.query('date > 85').reset_index(drop = True)
train[features[1:]] = train[features[1:]].fillna(f_mean)
train['action'] = (train['resp'] > 0).astype('int')

print('Converting...')
f_mean = f_mean.values
np.save('f_mean.npy', f_mean)

print('Finish.')

In [ ]:
# TODO: make GitHub GIST
# TODO: add as dataset
# TODO: add logging with verbose

import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
def get_callbacks(name, fold):
    return [
        ReduceLROnPlateau(monitor = 'val_r', factor = 0.1, patience = 10, verbose = 0, min_delta = 1e-4, mode = 'max'),
        ModelCheckpoint(f'{name}_{fold}.hdf5' , monitor = 'val_r', verbose = 0, save_best_only = True, 
                        save_weights_only = True, mode = 'max'),
        EarlyStopping(monitor = 'val_r', min_delta = 1e-4, patience = 20, mode = 'max', 
                      baseline = None, restore_best_weights = True, verbose = 0)
    ]

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

# Training

In [ ]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('linear')(x)

    model = tf.keras.models.Model(inputs = inp, outputs = out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss=tf.keras.losses.LogCosh(),
                  metrics=tfa.metrics.RSquare(name='r', dtype=tf.float32, y_shape=(1,)))
    
    return model

In [ ]:
name = "reg" #_reg_huber_r_"+str(pd.Timestamp.now())"
batch_size = 4096*4
hidden_units = [160, 80, 40, 20]
dropout_rates = [0.25, 0.5, 0.5, 0.5, 0.5]
learning_rate = 1e-3

# oof = np.zeros(len(train['action']))
# pgtss = PurgedGroupTimeSeriesSplit(n_splits = 5, group_gap = 20)

# for fold, (train_idx, test_idx) in enumerate(pgtss.split(train[features], groups=train['date'])):
#     start_time_fold = time()
    
#     X_train, X_val = train.loc[train_idx, features] , train.loc[test_idx, features]
#     y_train, y_val = train.loc[train_idx, 'resp'] * 100, train.loc[test_idx, 'resp'] * 100
    
#     model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate)

#     model.summary()
    
#     history = model.fit(
#         X_train,
#         y_train,
#         batch_size = batch_size,
#         validation_data = (X_val, y_val),
#         epochs = 250,
#         callbacks = get_callbacks(name, fold),
#         verbose = 1)
    
#     hist = pd.DataFrame(history.history)
#     print(f'[{str(datetime.timedelta(seconds = time() - start_time_fold))[0:7]}] Fold {fold} R^2:\t', hist['val_r'].max())

#     val_res = np.where(model.predict(X_val, batch_size = batch_size).ravel() > 0, 1, 0).astype(int)
#     oof[test_idx] += val_res
#     print("full u_score:", utility_score_bincount(train.date[test_idx], train.weight[test_idx], train.resp[test_idx], train.action[test_idx]))
#     print("model u_score:", utility_score_bincount(train.date[test_idx], train.weight[test_idx], train.resp[test_idx], val_res))
    
    
#     for i in np.arange(-0.2, 0.25, 0.05):
#         res = np.where(model.predict(X_val, batch_size = batch_size).ravel() > i, 1, 0).astype(int)
#         score = utility_score_bincount(train.date[test_idx], train.weight[test_idx], train.resp[test_idx], res)
#         print(f"> {i:.2f}% score: {score}")

#     model.save_weights(f'./reg333/{name}_{fold}.hdf5')
#     # Finetune 3 epochs on validation set with small learning rate
#     model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate / 100)
#     model.load_weights(f"./reg333/{name}_{fold}.hdf5")
#     model.fit(X_val, y_val, epochs = 3, batch_size = batch_size, verbose = 0)
#     model.save_weights(f"./reg333/{name}_{fold}_finetune.hdf5")
    
#     K.clear_session()
#     del model
#     rubbish = gc.collect()

In [ ]:
# score_oof = roc_auc_score(train['action'].values, oof)
# print(score_oof)

# Load Models

In [ ]:
USE_FINETUNE = True

In [ ]:
models_idx = [3,4]

models = []
for i in models_idx:
    clf = create_mlp(len(features), 1, hidden_units, dropout_rates, learning_rate)
    if USE_FINETUNE:
        clf.load_weights(f'../input/reg333/reg_{i}_finetune.hdf5')
    else:
        clf.load_weights(f'../input/reg333/reg_{i}.hdf5')
    models.append(clf)

In [ ]:
# f_mean = np.load('../input/js-nn-models/f_mean.npy')
f_mean = np.load('./f_mean.npy')

# Submitting

Just use two models to reduce running time.

In [ ]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
opt_th = 0.05
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = 0.
        pred = np.mean([clf(x_tt, training = False).numpy().item() for clf in models])
        # pred = models[3](x_tt, training = False).numpy().item()
        pred_df.action = np.where(pred > opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)